In [4]:
%load_ext autoreload
%autoreload 2
import feature_extract
import torch
# feature_extract.init_group()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
data_path = '/home/taylor/NFTResearchGIT/nft_research/Dino/images'
out_path = '/home/taylor/NFTResearchGIT/nft_research/Dino/features'
features = feature_extract.extract_feature_pipeline(model,data_path,out_path,use_cuda=True)


In [5]:
features = torch.load('/home/taylor/NFTResearchGIT/nft_research/dino/features/testfeat.pth')

# Print or use the loaded tensor as needed
print(features)

tensor([[ 0.0144, -0.0249,  0.0784,  ..., -0.0185,  0.0562,  0.0432],
        [ 0.0230,  0.0083, -0.0442,  ...,  0.0994,  0.0049,  0.0398],
        [-0.0012, -0.0027,  0.0017,  ...,  0.0230,  0.0237, -0.0110],
        ...,
        [ 0.0997, -0.0292,  0.0484,  ...,  0.0616,  0.0102,  0.0527],
        [-0.0669, -0.0039,  0.0519,  ...,  0.0811,  0.0245, -0.0259],
        [-0.0691,  0.0156,  0.0445,  ...,  0.0309,  0.0880,  0.0338]])


In [13]:
len(features)

120833

In [ ]:
euclidean_distance = torch.norm(features[0] - features[1], p=2) 

In [ ]:
euclidean_distance

In [18]:
labels = feature_extract.get_labels(data_path)

In [19]:
len(labels)

120833

In [20]:
import pandas as pd
features_list = features.tolist()

# Create a pandas DataFrame
data = {'Label': labels.tolist(), 'Features': features_list}
df = pd.DataFrame(data)
df

,Label,Features
0,0,"[0.014417114667594433, -0.024942265823483467, ..."
1,0,"[0.023010971024632454, 0.008338573388755322, -..."
2,0,"[-0.0012145170476287603, -0.002738840412348509..."
3,0,"[-0.012103280052542686, 0.07408665865659714, -..."
4,0,"[-0.08119715005159378, -0.03555002063512802, -..."
...,...,...
120828,470,"[-0.025555888190865517, 0.02635013870894909, 0..."
120829,470,"[0.014483450911939144, 0.06802485138177872, 0...."
120830,470,"[0.0996912494301796, -0.02922206185758114, 0.0..."
120831,470,"[-0.06689973920583725, -0.00392230274155736, 0..."


In [21]:
from scipy.spatial import distance
import itertools

# Assuming df is your pandas DataFrame with 'Label' and 'Features' columns

# Define a function to compute Euclidean distance between two feature vectors
def euclidean_distance(feature1, feature2):
    return distance.euclidean(feature1, feature2)

# Group the DataFrame by 'Label'
grouped = df.groupby('Label')

# Compute Euclidean distances for all pairs within each group
results = []
for label, group in grouped:
    feature_combinations = list(itertools.combinations(group['Features'], 2))
    distances = [euclidean_distance(pair[0], pair[1]) for pair in feature_combinations]
    if distances:
        results.append((label,sum(distances)/len(distances)))
    else:
        print(label,group)

# Print the computed distances
print(results)




59        Label                                           Features
15114     59  [0.07341117411851883, 0.017738036811351776, 0....
[(0, 0.9241125800345659), (1, 0.7870640680023046), (2, 0.07546301647519955), (3, 0.6043405200444767), (4, 1.2516698968544253), (5, 0.45162450310967395), (6, 0.21086513235368395), (7, 0.8212554634803055), (8, 0.4627970113993439), (9, 0.3801440654553813), (10, 0.3754330763700529), (11, 0.8877593032833073), (12, 0.6106548543092647), (13, 0.9409492820209624), (14, 0.4926428483052877), (15, 0.8524721350195693), (16, 1.224017048667783), (17, 0.4652287649688561), (18, 0.8755375103560851), (19, 0.9092054570817641), (20, 0.46520113443567135), (21, 0.421105475685858), (22, 0.6862405877195241), (23, 1.08104175959667), (24, 0.6810348519507889), (25, 0.631850946162354), (26, 0.8033951029869272), (27, 1.0184823642759322), (28, 0.9329648739060878), (29, 1.267817187392097), (30, 0.6278526335271485), (31, 0.7774769432654551), (32, 0.6488089507272742), (33, 0.860658968173912

In [22]:
# Create a DataFrame for average distances
average_distances_df = pd.DataFrame(results, columns=['Label', 'AverageDistance'])

# Merge the original DataFrame with average distances DataFrame on 'Label'
merged_df = pd.merge(df, average_distances_df, on='Label')
merged_df

,Label,Features,AverageDistance
0,0,"[0.014417114667594433, -0.024942265823483467, ...",0.924113
1,0,"[0.023010971024632454, 0.008338573388755322, -...",0.924113
2,0,"[-0.0012145170476287603, -0.002738840412348509...",0.924113
3,0,"[-0.012103280052542686, 0.07408665865659714, -...",0.924113
4,0,"[-0.08119715005159378, -0.03555002063512802, -...",0.924113
...,...,...,...
120827,470,"[-0.025555888190865517, 0.02635013870894909, 0...",0.999307
120828,470,"[0.014483450911939144, 0.06802485138177872, 0....",0.999307
120829,470,"[0.0996912494301796, -0.02922206185758114, 0.0...",0.999307
120830,470,"[-0.06689973920583725, -0.00392230274155736, 0...",0.999307


In [2]:
import pandas as pd
merged_df = pd.read_csv('average_distance.csv')
merged_df

,Label,Features,AverageDistance
0,0,"[0.014417114667594433, -0.024942265823483467, ...",0.924113
1,0,"[0.023010971024632454, 0.008338573388755322, -...",0.924113
2,0,"[-0.0012145170476287603, -0.002738840412348509...",0.924113
3,0,"[-0.012103280052542686, 0.07408665865659714, -...",0.924113
4,0,"[-0.08119715005159378, -0.03555002063512802, -...",0.924113
...,...,...,...
120827,470,"[-0.025555888190865517, 0.02635013870894909, 0...",0.999307
120828,470,"[0.014483450911939144, 0.06802485138177872, 0....",0.999307
120829,470,"[0.0996912494301796, -0.02922206185758114, 0.0...",0.999307
120830,470,"[-0.06689973920583725, -0.00392230274155736, 0...",0.999307


In [ ]:
# Find the row with the minimum average distance
q_df = merged_df.query('AverageDistance>0')
min_distance_row = merged_df.loc[q_df['AverageDistance'].idxmin()]

# Print the row with the minimum average distance
print(min_distance_row)

In [11]:
import os

# Directory path where the files are located
directory_path = '/home/taylor/NFTResearchGIT/nft_research/dino/images/val'

# Create a dictionary to store label-file associations
label_file_dict = {}

# Iterate through the files in the directory
for idx,filename in enumerate(sorted(os.listdir(directory_path))):
    # Extract the label from the file name (assuming the label is in the file name)
    print(idx,filename)
    # Check if the label exists in the merged_df DataFrame
    if idx in merged_df['Label'].values:
        # If yes, associate the file name with the label
        label_file_dict[idx] = filename

0 0010-editions
1 0n1-force
2 10ktf-gtags
3 10ktf-kagami
4 10ktf-stockroom
5 1337-skulls
6 2117-citizenship-bedu
7 3land-inventory
8 3landers
9 888innercirclethedrops
10 acclimatedmooncats
11 adidas-virtual-gear
12 adidascapsule
13 adworld
14 agoria-comp-nd-x-l-market
15 akidcalledbeast
16 akutar-accessories
17 akutar-mint-pass
18 akutars
19 allstarz-official
20 alphabetty-doodles
21 alphagatespacenation
22 amato-official
23 angel-banners
24 animetas
25 aopanda-party
26 ape-harbour-yachts
27 archive-edition
28 asa-eds
29 ashmetamorphosis
30 astarprince
31 atem-cards
32 athenaeum
33 atsnft
34 auf-reisen-2023-by-dennis-schmelz
35 azuki
36 azukielementals
37 babobunnyclub
38 badkidclub
39 banners-nft
40 basedheads
41 batzeditions
42 beanzofficial
43 betwixt-braves
44 biginc
45 bitcoin-miladys
46 bitgoatznft
47 blackandwhite-nft
48 bond-bears
49 bonepen-edition
50 bones-and-bananas
51 bonsai-zenft
52 boots-vn1ytsxkja
53 bored-and-dangerous
54 bored-ape-comic
55 bored-ape-kennel-club
56 bor

In [13]:
merged_df2['FileName'] = merged_df2['Label'].map(label_file_dict)

# Print the updated DataFrame
merged_df2


,Label,Features,AverageDistance,AverageFeatureVector,FileName
0,0,"[0.014417114667594433, -0.024942265823483467, ...",0.924113,"[0.012500456351387714, 0.017768271217521812, -...",0010-editions
1,0,"[0.023010971024632454, 0.008338573388755322, -...",0.924113,"[0.012500456351387714, 0.017768271217521812, -...",0010-editions
2,0,"[-0.0012145170476287603, -0.002738840412348509...",0.924113,"[0.012500456351387714, 0.017768271217521812, -...",0010-editions
3,0,"[-0.012103280052542686, 0.07408665865659714, -...",0.924113,"[0.012500456351387714, 0.017768271217521812, -...",0010-editions
4,0,"[-0.08119715005159378, -0.03555002063512802, -...",0.924113,"[0.012500456351387714, 0.017768271217521812, -...",0010-editions
...,...,...,...,...,...
120827,470,"[-0.025555888190865517, 0.02635013870894909, 0...",0.999307,"[-0.009466192685067654, 0.015365221817046405, ...",zapper-fi
120828,470,"[0.014483450911939144, 0.06802485138177872, 0....",0.999307,"[-0.009466192685067654, 0.015365221817046405, ...",zapper-fi
120829,470,"[0.0996912494301796, -0.02922206185758114, 0.0...",0.999307,"[-0.009466192685067654, 0.015365221817046405, ...",zapper-fi
120830,470,"[-0.06689973920583725, -0.00392230274155736, 0...",0.999307,"[-0.009466192685067654, 0.015365221817046405, ...",zapper-fi


In [5]:
import ast
merged_df['Features'] = merged_df['Features'].apply(ast.literal_eval)

In [6]:
import numpy as np
# Group the DataFrame by 'Label'
grouped = merged_df.groupby('Label')

# Compute average feature vector for each label
average_feature_vectors = []
for label, group in grouped:
    # Compute average feature vector for the current label
    avg_feature_vector = np.mean(group['Features'].tolist(), axis=0)
    average_feature_vectors.append((label, avg_feature_vector))
average_features_df = pd.DataFrame(average_feature_vectors, columns=['Label', 'AverageFeatureVector'])
merged_df2 = pd.merge(merged_df, average_features_df, on='Label')

In [10]:
merged_df2['FileName']

KeyError: 'FileName'

In [14]:
import pickle
with open('collection_features.pkl','wb') as f:
    pickle.dump(merged_df2,f)

In [17]:
import itertools
from scipy.spatial import distance
from tqdm import tqdm
# Assuming merged_df is your pandas DataFrame with 'Label' and 'AverageFeatureVector' columns

# Define a function to compute Euclidean distance between two feature vectors
def euclidean_distance(feature1, feature2):
    return distance.euclidean(feature1, feature2)

# Create combinations of labels
label_combinations = list(itertools.combinations(merged_df2['FileName'].unique(), 2))

# Compute distances between average feature vectors for every pair of labels
pairwise_distances = []
for label1, label2 in tqdm(label_combinations):
    feature_vector1 = merged_df2.loc[merged_df2['FileName'] == label1, 'AverageFeatureVector'].values[0]
    feature_vector2 = merged_df2.loc[merged_df2['FileName'] == label2, 'AverageFeatureVector'].values[0]
    dist = euclidean_distance(feature_vector1, feature_vector2)
    pairwise_distances.append((label1, label2, dist))

# Create a DataFrame for pairwise distances
pairwise_distances_df = pd.DataFrame(pairwise_distances, columns=['Label1', 'Label2', 'Distance'])

# Print the DataFrame with pairwise distances
pairwise_distances_df


100%|███████████████████████████████████| 110215/110215 [19:58<00:00, 91.96it/s]


,Label1,Label2,Distance
0,0010-editions,0n1-force,0.811090
1,0010-editions,10ktf-gtags,1.117590
2,0010-editions,10ktf-kagami,0.943008
3,0010-editions,10ktf-stockroom,0.755719
4,0010-editions,1337-skulls,0.809613
...,...,...,...
110210,youtherealmvp,zanozai,0.822462
110211,youtherealmvp,zapper-fi,0.904161
110212,zanozaartbank,zanozai,0.488142
110213,zanozaartbank,zapper-fi,0.698506


In [18]:
with open('pairwise_dist.pkl','wb') as f:
    pickle.dump(pairwise_distances_df,f)

In [20]:
# Find the row with the minimum average distance
min_distance_row = pairwise_distances_df.loc[pairwise_distances_df['Distance'].idxmax()]

# Print the row with the minimum average distance
print(min_distance_row)

Label1            redrex-og-pass-free
Label2      rug-radio-membership-pass
Distance                     1.463622
Name: 100083, dtype: object


In [23]:
sorted_df = pairwise_distances_df.sort_values(by='Distance')

# Get the top 50 rows with the lowest distances
top_50_lowest_rows = sorted_df.head(50)

# Print the top 50 rows
print(top_50_lowest_rows)

                       Label1                        Label2  Distance
82954           miladystation          miladystationclassic  0.044217
20300         bitcoin-miladys                        milady  0.056471
39210           crypto-phunks                   rockerpunks  0.069898
39145           crypto-phunks             official-v1-punks  0.102210
15820                   azuki               azukielementals  0.117522
91558       official-v1-punks                   rockerpunks  0.134884
90049           noble-gallery                  thememes6529  0.143221
56072         food-for-degens  open-edition-by-kevin-abosch  0.145597
95337           pixeladymaker              pixeladymaker-bc  0.157501
88285               nakamigos             official-v1-punks  0.176158
88350               nakamigos                   rockerpunks  0.179015
31010             clickcreate                 noble-gallery  0.179239
101571           robobots-nft                   rockerpunks  0.179396
39129           cryp